In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
from PIL import Image

# Define folder paths for reference images and target images
reference_image_folder = '/content/drive/MyDrive/Ocular_Disease/classes/pathological_myopia' # real image
target_image_folder = '/content/drive/MyDrive/Ocular_Disease/classified_output/Myopia' # synthetic (generated) image

# Load all images from both folders
reference_images = [os.path.join(reference_image_folder, img) for img in os.listdir(reference_image_folder)]
target_images = [os.path.join(target_image_folder, img) for img in os.listdir(target_image_folder)]

In [ ]:
# Import necessary libraries
from transformers import CLIPProcessor, CLIPModel
import torch

# Load the pre-trained CLIP model and processor from HuggingFace
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")  # Load the CLIP model (Vision Transformer)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")  # Load the CLIP processor for image preprocessing

# Function to extract features from an image
def get_image_features(image_path):
    """
    This function takes an image file path, processes the image,
    and extracts its feature representation using the CLIP model.
    
    Args:
    - image_path (str): Path to the image file.
    
    Returns:
    - features (tensor): The image feature representation as a tensor.
    """
    # Open and convert the image to RGB format (in case it's not in RGB)
    image = Image.open(image_path).convert('RGB')
    
    # Preprocess the image and return it as a tensor suitable for the CLIP model
    inputs = processor(images=image, return_tensors="pt")
    
    # Use the CLIP model to extract features without tracking gradients (for efficiency)
    with torch.no_grad():
        features = model.get_image_features(**inputs)  # Extract image features
    
    # Return the extracted features
    return features

# Dictionary to store the top 10 matches for each reference image
top_10_matches = {}


In [ ]:
# Compare each reference image with all target images
for ref_image in reference_images:  # Loop over each reference image
    # Extract features of the reference image
    reference_features = get_image_features(ref_image)  

    similarities = []  # List to store similarity scores between reference and target images
    
    for target_image in target_images:  # Loop over each target image
        # Extract features of the target image
        target_features = get_image_features(target_image)  
        
        # Compute cosine similarity between the reference and target image features
        similarity = torch.cosine_similarity(reference_features, target_features)
        
        # Append the target image and its similarity score to the list
        similarities.append((target_image, similarity.item()))

    # Sort the target images by their similarity score in descending order
    sorted_similarities = sorted(similarities, key=lambda x: x[1], reverse=True)[:10]
    
    # Store the top 10 most similar images for each reference image
    top_10_matches[ref_image] = sorted_similarities

# Print the top 10 matches for each reference image
for ref_image, matches in top_10_matches.items():  # Loop over reference images and their top matches
    print(f"Reference image: {ref_image}")  # Print the reference image
    for idx, (best_match, similarity) in enumerate(matches):  # Loop over the top matches
        print(f"Top {idx+1}: {best_match}, Similarity: {similarity}")  # Print each match with its similarity score
    print("\n")